<a href="https://colab.research.google.com/github/GReEeN17/Alghoritms_1_sem/blob/master/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0_%D0%B8_%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задача

Пусть дана простая выборка $X^{(2n)}$, известны конечные $EX_1=m$, $DX_1=\sigma^2$. Какая из двух несмещённых оценок $m$ более предпочтительна?

1. $\hat{m} = \frac{1}{2n}\sum\limits_{j=1}^{2n}X_j$

2. $\hat{m} = \frac{1}{n}\sum\limits_{j=1}^{n}X_j$

In [ ]:
import numpy as np
import plotly.graph_objects as go

from scipy.stats import norm
from statsmodels.graphics.gofplots import qqplot
from matplotlib import pyplot as plt

In [ ]:
mu = 10
sigma = 10

iter_size = 2
sample_size = 100000

x = norm(loc=mu, scale=sigma).rvs(size=[iter_size, sample_size], random_state=42)

In [ ]:
x = norm(loc=34, scale=sigma).rvs(size=[iter_size, sample_size], random_state=42).astype(int)
x


array([[38, 32, 40, ..., 38, 31, 35],
       [44, 22, 39, ..., 35, 42, 23]])

In [ ]:
mu_est_list = []
mean_mu_est_list = []
std_mu_est_list = []

In [ ]:
half_mu_est_list = []
half_mean_mu_est_list = []
half_std_mu_est_list = []

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[1, iter_size],
                         y=[mu, mu],
                         name="Заданное значение"))

x = norm(loc=mu, scale=sigma).rvs(size=[iter_size, sample_size], random_state=42)
mu_est_list = []
mean_mu_est_list = []

for i in range(iter_size):
    mu_est = x[i].mean()
    mu_est_list.append(mu_est)

    mean_mu_est = np.mean(mu_est_list)
    std_mu_est = np.std(mu_est_list)

    mean_mu_est_list.append(mean_mu_est)
    std_mu_est_list.append(std_mu_est)

fig.add_trace(go.Scatter(x=1 + np.arange(iter_size),
                         y=mean_mu_est_list,
                         name="Среднее оценки, N=2n"))

fig.update_xaxes(type="log")
fig.update_layout(title="Проверка несмещённости оценок",
                  xaxis_title="Размер выборки",
                  yaxis_title="Значение параметра")

In [ ]:
for i in range(iter_size):
    half_mu_est = x[i][:sample_size//2].mean()
    half_mu_est_list.append(half_mu_est)

    half_mean_mu_est = np.mean(half_mu_est_list)
    half_std_mu_est = np.std(half_mu_est_list)

    half_mean_mu_est_list.append(half_mean_mu_est)
    half_std_mu_est_list.append(half_std_mu_est)

fig.add_trace(go.Scatter(x=1 + np.arange(iter_size),
                         y=half_mean_mu_est_list,
                         name="Среднее оценки, N=n"))
fig.show()

In [ ]:
fig.show()

In [ ]:
std_all  = np.mean(std_mu_est_list)
std_half = np.mean(half_std_mu_est_list)

mean_all  = np.mean(mean_mu_est_list)
mean_half = np.mean(half_mean_mu_est_list)

In [ ]:
print(f'M={sample_size}, std=\'{std_all:6f}\', mean=\'{mean_all:6f}\'')
print(f'M={sample_size//2}, std=\'{std_half:6f}\', mean=\'{mean_half:6f}\'')
print(f'std_half/std_all=\'{std_half/std_all}\'')

M=12, std='28.704030', mean='10.142144'
M=6, std='40.497639', mean='10.499951'
std_half/std_all='1.4108694331059666'


## Точный доверительный интервал


$$
m \in I = \left(\overline{X} + z_{\alpha/2} \frac{10}{\sqrt{n}},
\overline{X} + z_{1-\alpha/2} \frac{10}{\sqrt{n}}\right).
$$

In [ ]:
def exact_solution(p: float, x: np.array) -> float:
    alpha = 1 - p
    return x.mean() + 10 * norm.ppf(alpha / 2) / np.sqrt(len(x)), \
           x.mean() + 10 * norm.ppf(1 - alpha / 2) / np.sqrt(len(x))

In [ ]:
solution = exact_solution

mu = 250
sigma = 10
p = 0.95

iter_size = 100
sample_size_list = [
    10,
    100,
    1000
]
color_map = {
    10: "red",
    100: "blue",
    1000: "green"
}

fig = go.Figure()
fig.add_trace(go.Scatter(x=[1, iter_size],
                         y=[mu, mu],
                         name="Заданное значение",
                         line={"color": "black"}))

for sample_size in sample_size_list:

    x = norm(loc=mu, scale=sigma).rvs(size=[iter_size, sample_size], random_state=42)
    left_side_list = []
    right_side_list = []

    for i in range(iter_size):
        left_side, right_side = solution(p, x[i])
        left_side_list.append(left_side)
        right_side_list.append(right_side)

    fig.add_trace(go.Scatter(x=1 + np.arange(iter_size),
                             y=left_side_list,
                             name=f"Нижняя граница, n = {sample_size}",
                             line={"color": color_map[sample_size]}))

    fig.add_trace(go.Scatter(x=1 + np.arange(iter_size),
                             y=right_side_list,
                             name=f"Верхняя граница, n = {sample_size}",
                             line={"color": color_map[sample_size]}))

fig.update_layout(title="Проверка доверительного интервала",
                  xaxis_title="Номер итерации",
                  yaxis_title="Значение параметра")
fig.show()

## Асимптотический доверительный интервал


$$
m \in I = \left(\overline{X} + z_{\alpha/2} \frac{S_X}{\sqrt{n}},
\overline{X} + z_{1-\alpha/2} \frac{S_X}{\sqrt{n}}\right).
$$

In [ ]:
def clt_solution(p: float, x: np.array) -> float:
    alpha = 1 - p
    return x.mean() - np.sqrt(np.var(x)) * norm.ppf(1 - alpha / 2) / np.sqrt(len(x)), \
           x.mean() - np.sqrt(np.var(x)) * norm.ppf(alpha / 2) / np.sqrt(len(x))

In [ ]:
solution = clt_solution

mu = 250
sigma = 10
p = 0.95

iter_size = 100
sample_size_list = [
    10,
    100,
    1000
]
color_map = {
    10: "red",
    100: "blue",
    1000: "green"
}

fig = go.Figure()
fig.add_trace(go.Scatter(x=[1, iter_size],
                         y=[mu, mu],
                         name="Заданное значение",
                         line={"color": "black"}))

for sample_size in sample_size_list:

    x = norm(loc=mu, scale=sigma).rvs(size=[iter_size, sample_size], random_state=42)
    left_side_list = []
    right_side_list = []

    for i in range(iter_size):
        left_side, right_side = solution(p, x[i])
        left_side_list.append(left_side)
        right_side_list.append(right_side)

    fig.add_trace(go.Scatter(x=1 + np.arange(iter_size),
                             y=left_side_list,
                             name=f"Нижняя граница, n = {sample_size}",
                             line={"color": color_map[sample_size]}))

    fig.add_trace(go.Scatter(x=1 + np.arange(iter_size),
                             y=right_side_list,
                             name=f"Верхняя граница, n = {sample_size}",
                             line={"color": color_map[sample_size]}))

fig.update_layout(title="Проверка доверительного интервала",
                  xaxis_title="Номер итерации",
                  yaxis_title="Значение параметра")
fig.show()

## Сравнение методик

In [ ]:
mu = 250
sigma = 10
p = 0.95

iter_size = 10000
test_stat = [{
    "sample_size": 3
}, {
    "sample_size": 10
}, {
    "sample_size": 100
}, {
    "sample_size": 1000
}]

solution_list = [{
    "name": "clt",
    "function": clt_solution
}, {
    "name": "exact",
    "function": exact_solution
}]

for test_element in test_stat:
    x = norm(loc=mu, scale=sigma).rvs(size=[iter_size, test_element["sample_size"]], random_state=42)

    for i in range(iter_size):
        for solution_element in solution_list:
            left_side, right_side = solution_element["function"](p, x[i])

            total_error_col = solution_element["name"] + "_total_error"
            total_interval_length_col = solution_element["name"] + "_total_interval_length"

            test_element[total_error_col] = test_element.get(total_error_col, 0) \
                                            + (1 if (mu < left_side or right_side < mu) else 0)
            test_element[total_interval_length_col] = test_element.get(total_interval_length_col, 0) \
                                                      + right_side - left_side

    for solution_element in solution_list:
        total_error_col = solution_element["name"] + "_total_error"
        mean_error_col = solution_element["name"] + "_mean_error"
        test_element[mean_error_col] = test_element[total_error_col] / iter_size

        total_interval_length_col = solution_element["name"] + "_total_interval_length"
        mean_interval_length_col = solution_element["name"] + "_mean_interval_length"
        test_element[mean_interval_length_col] = test_element[total_interval_length_col] / iter_size

column_description = [{
    "column": "sample_size",
    "description": "Размер выборки"
}, {
    "column": "exact_mean_error",
    "description": "Частота ошибок точного интервала"
}, {
    "column": "clt_mean_error",
    "description": "Частота ошибок асимптотического интервала"
}, {
    "column": "exact_mean_interval_length",
    "description": "Средняя длина точного интервала"
}, {
    "column": "clt_mean_interval_length",
    "description": "Средняя длина асимптотического интервала"
}]

test_data = pd.DataFrame(test_stat)
test_data[[el["column"] for el in column_description]] \
         .rename(columns={el["column"]: el["description"]
                          for el in column_description})

,Размер выборки,Частота ошибок точного интервала,Частота ошибок асимптотического интервала,Средняя длина точного интервала,Средняя длина асимптотического интервала
0,3,0.0476,0.2438,22.631715,16.422075
1,10,0.0565,0.0991,12.395901,11.430070
2,100,0.0506,0.0559,3.919928,3.890821
3,1000,0.0435,0.0430,1.239590,1.238703
